In [ ]:
# Clone github repository
!git clone --branch federated https://github.com/AlessandroMaini/federated-learning-project.git

In [ ]:
%cd federated-learning-project 

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import random
from data.cifar100_loader import get_cifar100_loaders
from models.prepare_model import get_frozen_dino_vits16_model
from tools.hyperparameter_tuning import run_grid_search
from eval import evaluate
from train import train
from sharding.Imbalance_division import data_division, print_classes_and_counts 
from FedAvg.averaging import averaging

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
flagTest = 1
type_sharding = 5

In [ ]:
train_loader, val_loader, test_loader, train_dataset, test_dataset = get_cifar100_loaders()
sharding_dataloaders, cardinality_datasets, sharding_dataloadersTest, cardinality_datasetsTest= data_division(train_dataset,test_dataset, type_sharding, flagTest)
criterion = nn.CrossEntropyLoss()

In [ ]:
collaborative_model = get_frozen_dino_vits16_model(device)



In [ ]:
start_round = 0
num_rounds = 2
best_test_acc = 0.0

hist_test_loss = []
hist_test_acc = []

In [ ]:
optimizer = optim.SGD(collaborative_model.parameters(), lr=0.01, momentum=0.95, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_rounds)

In [ ]:
# Collaborative model 
for round in range(start_round, start_round + num_rounds):
    clientsIndex = list(range(100)) 
    random.shuffle(clientsIndex)
    chosenClients = [sharding_dataloaders[i] for i in clientsIndex[:10]] 
    chosenCardinality = [cardinality_datasets[i] for i in clientsIndex[:10]]

    epochs = 4 
    averaging(chosenClients, chosenCardinality, collaborative_model, optimizer, device, epochs, criterion)
    test_loss, test_acc = evaluate(collaborative_model, test_loader, criterion, device)
    scheduler.step()

    hist_test_loss.append(test_loss)
    hist_test_acc.append(test_acc)

    print(f"Epoch {round+1}/{start_round + num_rounds}")
    print(f"  Test Loss:  {test_loss:.4f} | Test Acc:  {test_acc:.4f}")

In [ ]:
# Plot the training and test accuracy
plt.plot(test_acc, label='Test Accuracy')
plt.xlabel('Rounds')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
start_epoch = 0
num_epochs = 30

hist_test_loss = [[] for _ in range(100)]
hist_test_acc = [[] for _ in range(100)]


In [ ]:
# ragionevole solo quanto type_sharding è diverso da 1 
for client in range(100): 
    client_model = get_frozen_dino_vits16_model(device,type_sharding)
    optimizer = optim.SGD(client_model.parameters(), lr=0.01, momentum=0.95, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    for epoch in range(start_epoch, start_epoch + num_epochs):  
        train(client_model, sharding_dataloaders[client], optimizer, criterion, device)
        scheduler.step()
        test_loss, test_acc = evaluate(client_model, sharding_dataloadersTest[client], criterion, device)

        hist_test_loss[i].append(test_loss)
        hist_test_acc[i].append(test_acc)        